In [5]:
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from pydantic.v1 import BaseModel, Field
from langgraph.graph import StateGraph, START, END    
from langgraph.graph.message import add_messages 
from langchain_core.tools import tool      
import operator
from typing import List, Annotated, TypedDict, Union
from langgraph.checkpoint.memory import MemorySaver


import os
import json
from dotenv import load_dotenv
import asyncio
from langchain_groq import ChatGroq
from IPython.display import display, Markdown
from pprint import pprint
import requests

load_dotenv()

True

In [6]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "research-workshop-langgraph"
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"
os.environ["GROQ_API_KEY"] = "GROQ_API_KEY"

In [7]:
# --- AGENT INSTRUCTIONS ---
PLANNER_INSTRUCTIONS = "You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output 5 terms to query for."

SEARCH_INSTRUCTIONS = (
    "You are a research assistant. Given a search term, you search the web for that term and "
    "produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 "
    "words. Capture the main points. Write succintly, no need to have complete sentences or good "
    "grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the "
    "essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
)

WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

PUSH_INSTRUCTIONS = (
    "You are a member of a research team and will be provided with a short summary of a report. "
    "When you receive the report summary, you send a push notification to the user using your tool, "
    "informing them that research is complete, and including the report summary you receive"
)

In [9]:
# --- PYDANTIC SCHEMA ---
class WebSearchItem(BaseModel):
    reason: str = Field(description = "Your reason for why this search is important to your query")
    query: str = Field(description = "the search term to use for the web search")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description = "A list of web searches to perform")

class ReportData(BaseModel):
    short_summary: str = Field(description = "A short 2-3 sentence summary of the findings")
    markdown_report: str = Field(description = "The final report")
    follow_up_questions: List[str] = Field(description = "Suggested topics to research further")

In [10]:
# --- TOOLS ---

@tool
def web_search_tool(query: str) -> str:
    """Search the web for a given term. Use this for a research"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    request.post(pushover_url, data=payload)
    return "Notification Success"

In [12]:
# --- STATE DEFINITION ---

class ResearchState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    query: str
    search_plan: List[WebSearchItem]
    search_results: List[str]
    report: ReportData 

#### Note: Using llama-3.1-70b-versatile for high quality, 8b for simple tasks

In [13]:
# --- NODE IMPLEMENTATIONS (Using Groq) ---

llm_planner = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
llm_searcher = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
llm_writer = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
llm_notifier = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

In [16]:
async def planner_node(state: ResearchState):
    """Planner: Generates structure search plan using Groq."""
    planner = llm_planner.with_structured_output(WebSearchPlan)
    response = await planner.ainvoke([
        SystemMessage(content=PLANNER_INSTRUCTIONS),
        HumanMessage(content=f"Query: {state["query"]}")
    ])
    return {
        "search_plan": response.searches,
        "messages": AIMessage(content=f"Create search plan with {len(response.searches)} items.")
    }
    